In [ ]:
import numpy as np
import random
import pandas as pd
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, space_eval
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import (
    r2_score, mean_squared_error, mean_absolute_error,
    mean_absolute_percentage_error, mean_squared_log_error
)
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import pickle

In [ ]:
# 0) Reproducibility

random_seed = 42
np.random.seed(random_seed)
random.seed(random_seed)
rng = np.random.default_rng(random_seed)

In [ ]:
# 1) Data Load

csv_path = r"C:/Users/hangang/Desktop/Group 3_data_tra_lcocd.csv"
Data = pd.read_csv(csv_path, encoding='utf-8')
X = Data.drop('Chl-a', axis=1)
y = Data['Chl-a']

In [ ]:
# 2) Train/Test Split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=random_seed
)

In [ ]:
# 3) Metric Helper

def calculate_metrics(y_true, y_pred):
    y_true_clip = np.maximum(y_true, 0)
    y_pred_clip = np.maximum(y_pred, 0)
    r2 = r2_score(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true_clip, y_pred_clip)
    rmsle = np.sqrt(mean_squared_log_error(y_true_clip, y_pred_clip))
    return r2, rmse, mae, mse, mape, rmsle

In [ ]:
# 4) Hyperparameter Space

scalers = [StandardScaler(), MinMaxScaler(), RobustScaler()]
space = {
    'scaler': hp.choice('scaler', scalers),
    'n_estimators': hp.quniform('n_estimators', 10, 80, 1),
    'max_depth': hp.quniform('max_depth', 1, 7, 1),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.08),
    'gamma': hp.uniform('gamma', 0, 6),
    'min_child_weight': hp.quniform('min_child_weight', 1, 20, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.15, 1),
    'alpha': hp.uniform('alpha', 0, 6),
    'lambda': hp.uniform('lambda', 0, 6)
}


In [ ]:
# 5) Objective for TPE (3-fold CV)

def objective(params):
    scaler = params['scaler']
    n_estimators = int(params['n_estimators'])
    max_depth = int(params['max_depth'])
    learning_rate = float(params['learning_rate'])
    gamma = float(params['gamma'])
    min_child_weight = int(params['min_child_weight'])
    colsample_bytree = float(params['colsample_bytree'])
    alpha = float(params['alpha'])
    lambda_ = float(params['lambda'])

    X_train_scaled = scaler.fit_transform(X_train)

    model = xgb.XGBRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        gamma=gamma,
        min_child_weight=min_child_weight,
        colsample_bytree=colsample_bytree,
        reg_alpha=alpha,
        reg_lambda=lambda_,
        random_state=random_seed,
        n_jobs=-1
    )

    kf = KFold(n_splits=3, shuffle=True, random_state=random_seed)
    cv_scores = cross_val_score(
        model, X_train_scaled, y_train, cv=kf, scoring='neg_mean_squared_error'
    )
    mean_mse = -float(np.mean(cv_scores))
    return {'loss': mean_mse, 'status': STATUS_OK}

In [ ]:
# 6) Hyperparameter Optimization

trials = Trials()
best = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=200,
    trials=trials,
    rstate=rng
)
best_params = space_eval(space, best)

best_scaler = best_params['scaler']
best_n_estimators = int(best_params['n_estimators'])
best_max_depth = int(best_params['max_depth'])
best_learning_rate = float(best_params['learning_rate'])
best_gamma = float(best_params['gamma'])
best_min_child_weight = int(best_params['min_child_weight'])
best_colsample_bytree = float(best_params['colsample_bytree'])
best_alpha = float(best_params['alpha'])
best_lambda = float(best_params['lambda'])

In [ ]:
# 7) Train Best Model on Train Set

X_train_scaled = best_scaler.fit_transform(X_train)
X_test_scaled = best_scaler.transform(X_test)

best_model = xgb.XGBRegressor(
    n_estimators=best_n_estimators,
    max_depth=best_max_depth,
    learning_rate=best_learning_rate,
    gamma=best_gamma,
    min_child_weight=best_min_child_weight,
    colsample_bytree=best_colsample_bytree,
    reg_alpha=best_alpha,
    reg_lambda=best_lambda,
    random_state=random_seed,
    n_jobs=-1
)
best_model.fit(X_train_scaled, y_train)

In [ ]:
# 8) Evaluate

y_pred_train = best_model.predict(X_train_scaled)
y_pred_test = best_model.predict(X_test_scaled)

r2_train, rmse_train, mae_train, mse_train, mape_train, rmsle_train = calculate_metrics(y_train, y_pred_train)
r2_test,  rmse_test,  mae_test,  mse_test,  mape_test,  rmsle_test  = calculate_metrics(y_test,  y_pred_test)

print(f"\n[Best Scaler] {best_scaler.__class__.__name__}")
print("[Best Hyperparameters]")
print(f"  n_estimators      : {best_n_estimators}")
print(f"  max_depth         : {best_max_depth}")
print(f"  learning_rate     : {best_learning_rate:.4f}")
print(f"  gamma             : {best_gamma:.4f}")
print(f"  min_child_weight  : {best_min_child_weight}")
print(f"  colsample_bytree  : {best_colsample_bytree:.3f}")
print(f"  alpha             : {best_alpha:.4f}")
print(f"  lambda            : {best_lambda:.4f}")

print("\n[Train Metrics]")
print(f"  R2: {r2_train:.4f} | RMSE: {rmse_train:.4f} | MAE: {mae_train:.4f} | MSE: {mse_train:.4f} | MAPE: {mape_train:.4f} | RMSLE: {rmsle_train:.4f}")

print("\n[Test Metrics]")
print(f"  R2: {r2_test:.4f}  | RMSE: {rmse_test:.4f}  | MAE: {mae_test:.4f}  | MSE: {mse_test:.4f}  | MAPE: {mape_test:.4f}  | RMSLE: {rmsle_test:.4f}")

In [ ]:
# 9) Save Model (+ Scaler & Metadata)

model_save_path = r"C:/Users/hangang/Desktop/best_XGB_model_tra_lcocd.pkl"
with open(model_save_path, 'wb') as f:
    pickle.dump({
        'model': best_model,
        'scaler': best_scaler,
        'features': X.columns.tolist(),
        'random_seed': random_seed,
        'best_params': {
            'n_estimators': best_n_estimators,
            'max_depth': best_max_depth,
            'learning_rate': best_learning_rate,
            'gamma': best_gamma,
            'min_child_weight': best_min_child_weight,
            'colsample_bytree': best_colsample_bytree,
            'alpha': best_alpha,
            'lambda': best_lambda
        }
    }, f)

print(f"\nModel saved to: {model_save_path}")